In [1]:
!pip install langchain
!pip install langchain-community
!pip install langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 7.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 1.6 MB/s eta 0:00:00


In [2]:
import os
import yaml
with open('chatgpt-api-credentials.yml') as file:
    openai_key = yaml.safe_load(file)
os.environ['OPENAI_API_KEY'] = openai_key['OPENAI-API-KEY']

In [7]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

In [4]:
chatgpt = ChatOpenAI(model_name = 'gpt-4o',temperature=0.0)

In [13]:
reviews = [
    f"""
    Just received the Bluetooth speaker I ordered for beach outings, and it's fantastic.
    The sound quality is impressively clear with just the right amount of bass.
    It's also waterproof, which tested true during a recent splashing incident.
    Though it's compact, the volume can really fill the space.
    The price was a bargain for such high-quality sound.
    Shipping was also on point, arriving two days early in secure packaging.
    """,
    f"""
    Purchased a new gaming keyboard because of its rave reviews about responsiveness and backlighting.
    It hasn't disappointed. The keys have a satisfying click and the LED colors are vibrant,
    enhancing my gaming experience significantly. Price-wise, it's quite competitive,
    and I feel like I got a good deal. The delivery was swift, and it came well-protected,
    ensuring no damage during transport.
    """,
    f"""
    Ordered a set of wireless earbuds for running, and they've been a letdown.
    The sound constantly cuts out, and the fit is uncomfortable after only a few minutes of use.
    They advertised a 12-hour battery life, but I'm barely getting four hours.
    Considering the cost, I expected better quality and performance.
    They did arrive on time, but the positives end there. I'm already looking into a return.
    """,
    f"""
    The tablet stand I bought was touted as being sturdy and adjustable,
    but it's anything but. It wobbles with the slightest touch,
    and the angles are not holding up as promised. It feels like a breeze could knock it over.
    It was also pricier than others I've seen, which adds to the disappointment.
    It did arrive promptly, but what's the use if the product doesn't meet basic expectations?
    """,
    f"""
    Needed a new kitchen blender, but this model has been a nightmare.
    It's supposed to handle various foods, but it struggles with anything tougher than cooked vegetables.
    It's also incredibly noisy, and the 'easy-clean' feature is a joke; food gets stuck under the blades constantly.
    I thought the brand meant quality, but this product has proven me wrong.
    Plus, it arrived three days late. Definitely not worth the expense.
    """
]

In [17]:
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.pydantic_v1 import BaseModel,Field

class RAResponse(BaseModel):
    summary: str=Field(description="A brief summary of the user's review. It should be atmost 3 lines.")
    pros: list=Field(description="A list of atmost 3 bullet points showing positives that are implied in the review if any.")
    cons: list=Field(description="A list of atmost 3 bullet points showing negatives that are implied in the review if any.")
    review: str=Field(description="Define if the user's review is positive, negative or neutral")
    emotions: list=Field(description="A list of 3 to 5 emotions expressed by the customer in the review.")
    email: str=Field(description="A detailed email to the customer based on the sentiment")
    
output_parser = PydanticOutputParser(pydantic_object=RAResponse)
    

In [18]:
prompt_text = """
Act as a helpful AI assistant. You have to perform sentiment analysis on customer reviews and the output has to be in the following format.
Output format: {format_instructions}
Also remember to write a detailed email response for the email field based on these conditions:
- email should be addressed to Dear Customer and signed with Service Agent
- thank them if the review is positive or neutral
- apologize if the review is negative
Review: {review}
"""
prompt = PromptTemplate(
    template=prompt_text,
    input_variables=["review"],
    partial_variables={"format_instructions":output_parser.get_format_instructions()},
)


In [19]:
chain=(
    prompt
    |
    chatgpt
    |
    output_parser
)

In [23]:
response = chain.map().invoke([{"review": review } for review in reviews])
response[0]

RAResponse(summary='The customer is very satisfied with the Bluetooth speaker, praising its sound quality, waterproof feature, and timely shipping.', pros=['Impressively clear sound quality with the right amount of bass', 'Waterproof feature works well', 'Compact yet powerful volume', 'Affordable price for high-quality sound', 'Timely and secure shipping'], cons=[], review='positive', emotions=['satisfaction', 'happiness', 'impressed', 'contentment', 'gratitude'], email="Dear Customer,\n\nThank you so much for your wonderful review! We are thrilled to hear that you are enjoying your new Bluetooth speaker. It's great to know that the sound quality, waterproof feature, and compact design have met your expectations. We are also pleased that the shipping was timely and secure.\n\nYour satisfaction is our top priority, and we are grateful for your positive feedback. If you have any further questions or need assistance, please do not hesitate to reach out.\n\nThank you once again for choosin

In [24]:
response[0].dict()

{'summary': 'The customer is very satisfied with the Bluetooth speaker, praising its sound quality, waterproof feature, and timely shipping.',
 'pros': ['Impressively clear sound quality with the right amount of bass',
  'Waterproof feature works well',
  'Compact yet powerful volume',
  'Affordable price for high-quality sound',
  'Timely and secure shipping'],
 'cons': [],
 'review': 'positive',
 'emotions': ['satisfaction',
  'happiness',
  'impressed',
  'contentment',
  'gratitude'],
 'email': "Dear Customer,\n\nThank you so much for your wonderful review! We are thrilled to hear that you are enjoying your new Bluetooth speaker. It's great to know that the sound quality, waterproof feature, and compact design have met your expectations. We are also pleased that the shipping was timely and secure.\n\nYour satisfaction is our top priority, and we are grateful for your positive feedback. If you have any further questions or need assistance, please do not hesitate to reach out.\n\nTha

In [26]:
import pandas as pd

pd.DataFrame(r.dict() for r in response)

,summary,pros,cons,review,emotions,email
0,The customer is very satisfied with the Blueto...,[Impressively clear sound quality with the rig...,[],positive,"[satisfaction, happiness, impressed, contentme...","Dear Customer,\n\nThank you so much for your w..."
1,Purchased a new gaming keyboard because of its...,"[Satisfying key click, Vibrant LED colors, Com...",[],positive,"[satisfaction, happiness, excitement, contentm...","Dear Customer,\n\nThank you for taking the tim..."
2,"Ordered a set of wireless earbuds for running,...",[Arrived on time],"[Sound constantly cuts out, Uncomfortable fit,...",negative,"[disappointed, frustrated, let down, dissatisf...","Dear Customer,\n\nWe sincerely apologize for t..."
3,The tablet stand is not sturdy or adjustable a...,[Arrived promptly],"[Wobbles with the slightest touch, Angles are ...",negative,"[disappointment, frustration, dissatisfaction,...","Dear Customer,\n\nWe sincerely apologize for t..."
4,The customer is dissatisfied with the kitchen ...,[],"[Struggles with tougher foods, Incredibly nois...",negative,"[disappointment, frustration, dissatisfaction,...","Dear Customer,\n\nWe sincerely apologize for t..."
